In [1]:
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main amd64 Packages [875 kB]


In [2]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (81.0.4044.122-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [3]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [4]:
!pip install -U selenium

Requirement already up-to-date: selenium in /usr/local/lib/python3.6/dist-packages (3.141.0)


In [5]:
!pip install beautifulsoup4
!pip install lxml

# ***About Glassdoor.com*** 
![alt text](https://www.glassdoor.com/app/static/img/partners/fb/logo-1200x630.png?v=721f9cswl)

   Glassdoor is a website where current and former employees anonymously review companies. Glassdoor also allows users to anonymously submit and view salaries as well as search and apply for jobs on its platform.

   Glassdoor produces reports based upon the data its anonymous and, in many cases, managers generate positive reviews for their organizations through posts. These reports have been on topics including work-life balance, CEO pay-ratios, lists of the best office places and cultures,and the accuracy of corporate job searching maxims. Data from Glassdoor has also been used by outside sources to produce estimates on the effects of salary trends and changes on corporate revenues.Glassdoor also puts the conclusions of its research of other companies towards its own company policies. In 2015 Tom Lakin produced the first study of Glassdoor in the United Kingdom, concluding that Glassdoor is regarded by users as more trustworthy sources of information than career guides or official company documents.



#  ***Logic of the scraping*** 



1.   In the first step extract links of every job listed from all the pages of the website.
2.   Then we use selenium and beautiful soup(BS4) to to extract the page source.
* Page source has a JSON which contains everything except Rating. 

* We use bs4 to extract rating from the mentioned tag and use class ='desc' to extract job descriptions. The reason for using 'desc' class is that it contains only text field and text has no extraneous HTML tags.
3. A fair bit of warning don't put too many requests on the server as it may block your IP. Run this code wisely.




In [0]:
#importing libraries
from selenium import webdriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
import time
from selenium.common.exceptions import NoSuchElementException , ElementClickInterceptedException
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import lxml
import re
import pandas as pd
import json
import time

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [0]:
def CheckUnique(links):

    
        count_dict = {link:links.count(link) for link in links if links.count(link) > 1}
        clean_links = [link for link in links if link not in list(count_dict.keys())]
        return clean_links

def ScrapeLinks(driver , start_pg , end_pg ,total_pages):

        #Total pages are the total web pages on the website not the total you want to scrape
        if end_pg > total_pages:
            print("Enter valid Page Indexes")
        br = driver
        scr_links = []
        url_head = 'https://www.glassdoor.co.in'
        #br = self.Initialize_Driver()
        for pg_num in range(start_pg, end_pg+1):
            url = 'https://www.glassdoor.co.in/Job/new-york-data-scientist-jobs-SRCH_IL.0,8_IC1132348_KO9,23_IP'+str(i)+'.htm?fromAge=14'
            br.get(url)
            br.implicitly_wait(5)

            soup = BeautifulSoup(br.page_source , 'lxml')

            #print(f"Scraping Page Number : {pg_num}")
            for a_tag in soup.find_all('a' , href = True):
                pattern = re.compile('/partner/jobListing')
                if pattern.match(str(a_tag['href'])):
                    scr_links.append(url_head + str(a_tag['href']))
        
        op_links = CheckUnique(scr_links)
        if len(op_links) == ((end_pg-start_pg) +1)*30 :
            print("Scraping of links is completed")
            return op_links
        

        

def ScrapeData(num_jobs,driver , links):

        br = driver
        #data_df = df
        #cleaned_links = ScrapeLinks(br)
        #if data_df is None:
        data_df = pd.DataFrame(columns = ['Job_title' , 'Company' , 'State' , 'City' , 'Min_Salary' ,'Max_Salary' ,'Job_Desc' , 
                                         'Industry'  , 'Rating' , 'Date_Posted' , 'Valid_until' , 'Job_Type' ,
                                          ])
        
        start = time.time()
        print("Gathering Information")
        for job in range(num_jobs):

            #print(f'Extracting information about job : {job}')
            br.get(links[job])
            br.implicitly_wait(10)

            soup = BeautifulSoup(br.page_source , 'lxml')

            try:
                
                #The rest of the data is available in scrip = 'application/ld+json' i.e in  json file
                json_file = soup.find('script' , {'type':'application/ld+json'})
                op_json = json.loads(str(json_file.text) , strict = False)
                
                #title
                try:
                    title = op_json['title']
                    #print(title)
                except:
                    title  = None
                
                #Dateposted
                try:
                    post_date = op_json['datePosted']
                except:
                    post_date = None
                
                #type
                try:
                    job_type = op_json['employmentType']
                except:
                    job_type = None

                #Annual Salary Range
                try:
                    min_salary = int(op_json['estimatedSalary']['value']['minValue'])
                    max_salary = int(op_json['estimatedSalary']['value']['maxValue'])
                except:
                    min_salary = -1
                    max_salary = -1
                
                #Job Posting validity date(Y-M-D)
                try:
                    valid_date = op_json['validThrough']
                except:
                    valid_date = None
                
                #Industry
                try:
                    industry = op_json['industry']
                except:
                    industry = None
                
                #Location
                try:
                    city = op_json['jobLocation']['address']['addressLocality']
                    state = op_json['jobLocation']['address']['addressRegion']
                except:
                    city , state = None , None

                #Company
                try:
                    company = op_json['hiringOrganization']['name']
                except:
                    company = None
                
                #Let's get Description
                try:
                    desc = soup.find(class_ = 'desc').text
                except:
                    desc = None
                try:
                    rating = soup.find('span' , {'class' : 'css-1pmc6te e11nt52q3'}).text.replace('★' , '')
                
                except:
                    rating = None


            
                data_df = data_df.append({'Job_title' : title,
                                      'Company' : company,
                                      'State' : state,
                                      'City' : city,
                                      'Min_Salary':min_salary,
                                      'Max_Salary':max_salary,
                                      'Job_Desc' : desc,
                                      'Industry':industry,
                                      'Rating':rating,
                                      'Date_Posted' : post_date,
                                      'Valid_until' : valid_date,
                                      'Job_Type' :job_type} , ignore_index = True)
            except:
                pass
        
        #driver.close()
        print(f"Scraping Completed for {data_df.shape[0]} jobs")
        print(f"Time Required : {time.time() - start} seconds")
        
        return data_df
        

    

In [17]:
#Please Specify number of jobs as an excessive number of requests made by the computer Ip can potentially block you
#glassdoor has 30 listing per page, so you can calculate manually how much is the total amount of jobs present i.e (num_pages*30)
num_jobs = 150
start_pg =1 
end_pg = 30
total_pages = 30

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
driver = webdriver.Chrome('chromedriver' , options = options)

links = ScrapeLinks(driver,start_pg,end_pg , 30)


Scraping of links is completed


In [19]:
links[:5]

['https://www.glassdoor.co.in/partner/jobListing.htm?pos=0&ao=37049&s=58&guid=&src=GD_JOB_AD&vt=w&cs=1i_7f4cef43&cb=1588782263572&jobListingId=3353360125',
 'https://www.glassdoor.co.in/partner/jobListing.htm?pos=0&ao=8095&s=58&guid=&src=GD_JOB_AD&vt=w&cs=1i_1fb5cf82&cb=1588782263572&jobListingId=3331413851',
 'https://www.glassdoor.co.in/partner/jobListing.htm?pos=0&ao=8095&s=58&guid=&src=GD_JOB_AD&vt=w&cs=1i_77680cf0&cb=1588782263572&jobListingId=3560810903',
 'https://www.glassdoor.co.in/partner/jobListing.htm?pos=0&ao=4120&s=58&guid=&src=GD_JOB_AD&vt=w&cs=1i_a2726c63&cb=1588782263572&jobListingId=3452858287',
 'https://www.glassdoor.co.in/partner/jobListing.htm?pos=0&ao=154438&s=58&guid=&src=GD_JOB_AD&vt=w&ea=1&cs=1i_8368af9d&cb=1588782263572&jobListingId=2836002558']

In [23]:
total_jobs = 30*(end_pg - start_pg+1)
f_df = pd.DataFrame(columns = ['Job_title' , 'Company' , 'State' , 'City' , 'Min_Salary' ,'Max_Salary' ,'Job_Desc' , 
                                         'Industry'  , 'Rating' , 'Date_Posted' , 'Valid_until' , 'Job_Type' ,
                                          ])
dummy_df  = ScrapeData(10 , driver , links)
#Exctract 150 jobs every 1 mins 
#for i in range(total_jobs//num_jobs):
     #df = ScrapeData(num_jobs , driver , links[num_jobs*i : num_jobs*(i+1)])
     #f_df = f_df.append(df)
     #time.sleep(30)


Gathering Information
Scraping Completed for 10 jobs
Time Required : 40.4912006855011 seconds


In [0]:
#Save the file to csv
f_df.to_csv('Data_Job_NY' , index = False)

In [0]:
SF_link  ='https://www.glassdoor.co.in/Job/san-francisco-data-scientist-jobs-SRCH_IL.0,13_IC1147401_KO14,28_IP'+str(pg_num)+'.htm?fromAge=14'
NY_Link = 'https://www.glassdoor.co.in/Job/new-york-data-scientist-jobs-SRCH_IL.0,8_IC1132348_KO9,23.htm?fromAge=14'